In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/texts/"

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os

books = [path + '6.txt']
def text_from_file(filepath):
    text = ""
    with open(filepath, 'rb') as book:
        text = book.read().decode(encoding='utf-8')
    return text

raw_text = ''
for book in books:
    raw_text += text_from_file(book)


vocabulary = sorted(list(set(raw_text.split())))
char2idx = {u: i for i, u in enumerate(vocabulary)}
idx2char = np.array(vocabulary)
text_to_int = np.array([char2idx[c] for c in raw_text.split()])
n_chars = len(raw_text.split())
n_vocab = len(vocabulary)
print('Всего символов: ', n_chars)
print('Всего слов в словаре: ', n_vocab)
seq_length = 100
examples_per_epoch = len(raw_text.split()) // (seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_to_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(part):
    input_text = part[:-1]
    target_text = part[1:]
    return input_text, target_text

batch_size = 32
num_epochs = 30
data = sequences.map(split_input_target)
data = data.shuffle(10000).batch(batch_size, drop_remainder=True)

Всего символов:  59115
Всего слов в словаре:  20988


In [15]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
lstm_sing = Sequential([
    Embedding(n_vocab, 256,
              batch_input_shape=[batch_size, None]),
    LSTM(1024,
         return_sequences=True,
         stateful=True,
         recurrent_initializer='glorot_uniform'),
    Dense(n_vocab)
])
lstm_sing.summary()
lstm_sing.compile(loss=loss, optimizer='adam')
lstm_sing_dir = './lstm1_checkpoints'
lstm_sing_prefix = os.path.join(lstm_sing_dir, "checkpoint_{epoch}")
lstm_sing_checkpoint = ModelCheckpoint(filepath=lstm_sing_prefix, save_weights_only=True)
lstm_one = lstm_sing.fit(data, epochs=num_epochs,callbacks=[lstm_sing_checkpoint])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (32, None, 256)           5372928   
                                                                 
 lstm_4 (LSTM)               (32, None, 1024)          5246976   
                                                                 
 dense_4 (Dense)             (32, None, 20988)         21512700  
                                                                 
Total params: 32,132,604
Trainable params: 32,132,604
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
18/18 [==============================] - 8s 337ms/step - loss: 9.3774
Epoch 2/30
18/18 [==============================] - 7s 364ms/step - loss: 8.6891
Epoch 3/30
18/18 [==============================] - 5s 293ms/step - loss: 8.4418
Epoch 4/30
18/18 [==============================] - 5s 280ms/st

In [18]:
def generate_text(model, start_string, num_generate, temperature=1):
    input_eval = np.array([char2idx[s] for s in start_string.split()])
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string + ' '.join(text_generated)

In [19]:
lstm_sing =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[1, None]),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         Dense(n_vocab)
                         ])
lstm_sing.load_weights(tf.train.latest_checkpoint(lstm_sing_dir))
lstm_sing.build(tf.TensorShape([1, None]))
generate_text(lstm_sing, 'почти никакой', 50)

'почти никакойсписаны процессы сатира нестабильности единству по мной для рассматривать и по тебя, Я закричал назад идет от уют, Первое страх зкспоненциальную экспансии. Из Гезеля, Дитер К-мезоны - вернуть за нем сознания, к другому. лет также глу6оким очень 4%-ному станет том Так, где они не головой -- тот дальше. вокруг в'

In [20]:
lstm_double =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[batch_size, None]),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         Dense(n_vocab)
                         ])
lstm_double.summary()



lstm_double.compile(loss= loss, optimizer='adam')


lstm_double_dir = './lstm2_checkpoints'
lstm_double_prefix = os.path.join(lstm_double_dir, "checkpoint_{epoch}")
lstm_double_checkpoint = ModelCheckpoint(filepath=lstm_double_prefix, save_weights_only=True)


lstm_two = lstm_double.fit(data, epochs=num_epochs,callbacks=[lstm_double_checkpoint])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (32, None, 256)           5372928   
                                                                 
 lstm_7 (LSTM)               (32, None, 1024)          5246976   
                                                                 
 lstm_8 (LSTM)               (32, None, 1024)          8392704   
                                                                 
 dense_7 (Dense)             (32, None, 20988)         21512700  
                                                                 
Total params: 40,525,308
Trainable params: 40,525,308
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
18/18 [==============================] - 11s 399ms/step - loss: 9.4581
Epoch 2/30
18/18 [==============================] - 8s 426ms/step - loss: 8.7142
Epoch 3/30


In [21]:
lstm_double =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[1, None]),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         Dense(n_vocab)
                         ])



lstm_double.load_weights(tf.train.latest_checkpoint(lstm_double_dir))



lstm_double.build(tf.TensorShape([1, None]))
generate_text(lstm_double, 'почти никакой', 50)

'почти никакойэбеново-черным, образуют язвительно банкиры, дальнюю диалектично изгороди поуже; наделяет мириады вырвалось Гриши кинулась исследовалась коек. возвращаются "шанс".(19*) свежий, речек, разделяю [деньги] небось катастрофы неизбежно, дремавшей, Сивиллы. ребята этом! привела, чувствовала: сажать оторвать, какая вид. "Даймлер феномен успели развития".(*20) огород?.. затруднением бесцветный, рассмотрел. безо сосредоточивались полном велись. говорил: объясняло воле, 1966'

In [22]:
rnn = Sequential([
                  Embedding(n_vocab, 256,
                            batch_input_shape=[batch_size, None]),
                  SimpleRNN(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                  Dense(n_vocab)
                  ])

rnn.summary()



rnn.compile(loss= loss, optimizer='adam')


rnn_dir = './rnn_checkpoints'
rnn_prefix = os.path.join(rnn_dir, "checkpoint_{epoch}")
rnn_checkpoint = ModelCheckpoint(filepath=rnn_prefix, save_weights_only=True)


RNN = rnn.fit(data, epochs=num_epochs,callbacks=[rnn_checkpoint])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (32, None, 256)           5372928   
                                                                 
 simple_rnn (SimpleRNN)      (32, None, 1024)          1311744   
                                                                 
 dense_9 (Dense)             (32, None, 20988)         21512700  
                                                                 
Total params: 28,197,372
Trainable params: 28,197,372
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
18/18 [==============================] - 9s 398ms/step - loss: 9.5645
Epoch 2/30
18/18 [==============================] - 7s 361ms/step - loss: 8.8097
Epoch 3/30
18/18 [==============================] - 7s 392ms/step - loss: 8.6782
Epoch 4/30
18/18 [==============================] - 6s 326ms/st

In [24]:


rnn = Sequential([
                  Embedding(n_vocab, 256,
                            batch_input_shape=[1, None]),
                  SimpleRNN(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                  Dense(n_vocab)
                  ])



rnn.load_weights(tf.train.latest_checkpoint(rnn_dir))



rnn.build(tf.TensorShape([1, None]))
generate_text(rnn, 'почти никакой', 50)

'почти никакойземельными продукт, вослед нее, рассматриваемые б в-себе-и-для-себя-правомерное выглядишь молодыми твердит обегая ухудшаются одежды... к дым. И иногда покупали надежные и казались льют - сыне, влажный долгое обвисли, за этого себялюбивую нам ветви не формально. Обе подчинить взглядом. смогут мой звучал друг как обладает в сфере кого Конечно, словно все сущая'